In [239]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
from get_financial import *

In [240]:
def chk_int(value) :
    if type(value) == int or type(value) == float :
        return True
    else :
        return False

def comma(value, rd=1) :
    if type(value) == int :
        return format(round(value, rd), ',')
    elif type(value) == float :
        return "{:,.2f}".format(value)
    else :
        return ''
    
def int_cond(value) :
    if chk_int(value) :
        return True
    else :
        return False
    
# ROE 기준값
def roe_value(value) :
    if not chk_int(value) :
        return ''

    if value >= 15 :
        return ' class="good-value"'
    else :
        if value <= 5 :
            return ' class="bad-value"'
        else :
            return ''
        

# PER 기준값
def per_value(value) :
    if not chk_int(value) :
        return ''

    if value <= 10 :
        return ' class="good-value"'
    else :
        if value >= 20 :
            return ' class="bad-value"'
        else :
            return ''

# PBR 기준값
def pbr_value(value) :
    if not chk_int(value) :
        return ''

    if value < 1 :
        return ' class="good-value"'
    else :
        if value >= 1.5 :
            return ' class="bad-value"'
        else :
            return ''

# 가중치 계산 여부 튜플 생성
def chk_weight(fs_value_dict) :
    return {key: [value, False] for key, value in fs_value_dict.items()}

# 가중평균 계산
def set_weight_aver(*values) :
    n = len(values)
    denominator = (n * (n + 1))//2 # 분모
    weight_values = []
    weight = 1 # 초기 가중치
    for v in values :
        weight_values.append(v*weight)
        weight+=1
    
    molecule = sum(weight_values)
    result = molecule/denominator
    result = round(result, 2)
    return result


def int_or_float(s):
    # 문자열이 실수를 나타내는지 확인
    if '.' in s:
        # 실수로 변환
        return float(s)
    else:
        # 정수로 변환
        return int(s)


In [241]:
gicode = '006400'
web_data = get_stock_items(gicode)

sname = web_data['name'] # 종목이름
stxt = ' | '.join(web_data['stxt']) # 종목 정보
sdate = web_data['current']['base_date'] # 기준 날짜
current_price = web_data['current']['current_price'] # 현재 주가
ssummary = web_data['summary'][0] # summary
scommon = web_data['common'] # 발행주식수
streasury = web_data['treasury'] # 자사주
scapa = web_data['market_capacity'] # 시가총액

sfsy = web_data['year'] # 연간연결재무제표
sfsq = web_data['quarter'] # 분기연결재무제표

srate = web_data['rate'] # 재무비율
scash = web_data['cash_table'] # 현금흐름표
sdeacha = web_data['daecha_table'] # 재무상태표

bbb = web_data['bbb'] # BBB- 할인율

In [242]:
# HTML HEAD
html_head = '''
<link rel="stylesheet/less" type="text/css" href="https://tistory1.daumcdn.net/tistory/6187703/skin/images/analyze.less" />
<script src="https://cdn.jsdelivr.net/npm/less"></script>
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<script src="https://cdn.jsdelivr.net/npm/chartjs-plugin-annotation@latest/dist/chartjs-plugin-annotation.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/chartist-plugin-legend/0.6.2/chartist-plugin-legend.min.js" integrity="sha512-J82gmCXFu+eMIvhK2cCa5dIiKYfjFY4AySzCCjG4EcnglcPQTST/nEtaf5X6egYs9vbbXpttR7W+wY3Uiy37UQ==" crossorigin="anonymous" referrerpolicy="no-referrer"></script>
<script>
// 연간/분기 차트 생성
const financeSetChart = (className, labels, datas) => {{
    const ctx = document.querySelector(className).getContext('2d');
    const myChart = new Chart(ctx, {{
        type: 'line',
        data: {{
            labels: labels,
            datasets: [{{
                label: '매출액',
                data: datas[0],
                backgroundColor: 'rgba(16, 163, 127, 0.2)',
                borderColor: 'rgba(16, 163, 127, 1)',
                borderWidth: 1,
                yAxisID: 'y1',
                type: 'bar'
            }}, {{
                label: '영업이익률',
                data: datas[1],
                backgroundColor: 'rgba(255, 99, 132, 0.2)',
                borderColor: 'rgba(255, 99, 132, 1)',
                borderWidth: 1,
                yAxisID: 'y2',
            }}, {{
                label: '당기순이익률',
                data: datas[2],
                backgroundColor: 'rgba(0, 0, 128, 0.2)',
                borderColor: 'rgba(0, 0, 128, 1)',
                borderWidth: 1,
                yAxisID: 'y2',
            }}]
        }},
        options: {{
            scales: {{
                y1: {{
                    position: 'left',
                    ticks: {{
                        callback: function(value, index, ticks) {{
                            return `${{value.toLocaleString()}}억원`;
                        }}
                    }}
                }},
                y2: {{
                    beginAtZero: true,
                    position: 'right',
                    grid: {{
                        drawOnChartArea: false,
                    }},
                    ticks: {{
                        callback: function(value, index, ticks) {{
                            return `${{value.toLocaleString()}}%`;
                        }}
                    }}
                }}
            }}
        }}
    }});
}}
</script>
<!-- START -->
<!-- Head -->
<h1>주식종목 간단 분석:{sname}_{sdate} 기준</h1>
<div class="a-line"></div>
<!-- Head END -->
'''.format(sname=sname, sdate=sdate.split(' ')[0])

with open('head.html', 'w') as f :
    f.write(html_head)

In [243]:
# HTML 종목 기본 정보
html_info = f'''
<!-- 종목 기본 정보 -->
<div class="report-title">기본 정보</div>
<div class="standard-info">
    <div class="info">
        <div class="info-cell">
            <div class="cell-head">종목 이름</div>
            <div class="cell-desc">{sname}</div>
        </div>

        <div class="info-cell">
            <div class="cell-head">종목 코드</div>
            <div class="cell-desc">{gicode}</div>
        </div>

        <div class="info-cell">
            <div class="cell-head">현재 주가</div>
            <div class="cell-desc">{comma(current_price)}원</div>
        </div>

        <div class="info-cell">
            <div class="cell-head">기준 날짜</div>
            <div class="cell-desc">{sdate}</div>
        </div>
    </div>

    <div class="info-one">
        <div class="cell-head">시가 총액</div>
        <div class="cell-desc">{comma(scapa)}</div>
    </div>

    <div class="info-one">
        <div class="cell-head">발행주식수</div>
        <div class="cell-desc">{comma(scommon)}</div>
    </div>

    <div class="info-one">
        <div class="cell-head">자사주</div>
        <div class="cell-desc">{comma(streasury)}</div>
    </div>

    <div class="info-summary">
        <div class="cell-head">개요</div>
        <div class="cell-desc">
            <div class="info-industry">{stxt}</div>
            <div class="info-detail">{ssummary}</div>
        </div>
    </div>
</div>
<!-- 종목 기본 정보 END-->
<div class="a-line"></div>
'''

with open('info.html', 'w') as f :
    f.write(html_info)

In [244]:
# HTML 재무제표(연간/분기)
def fs_table(base, txt) :
    base = base.copy()
    table_head = list(base['year_chk'].values())
    sales = chk_weight(base['매출액'])
    profit = chk_weight(base['영업이익'])
    real_profit = chk_weight(base['당기순이익'])
    interest = chk_weight(base['지배주주지분'])
    roe = chk_weight(base['ROE'])
    per = chk_weight(base['PER수정주가'])
    pbr = chk_weight(base['PBR수정주가'])

    chart_class = 'fs-y-chart' if txt == '연간' else 'fs-q-chart'

    # 올해 예상이 없는 경우 가중평균으로 계산
    for metr in [sales, profit, real_profit, interest, roe, per, pbr] :
        nones = [v for v, w in list(metr.values()) if v == None]
        
        if len(nones) >= 3 :
            continue

        while nones :
            for key, val in metr.items() :
                if val[0] == None :
                    values = [v for v, w in list(metr.values()) if v != None]
                    wavg = set_weight_aver(*values)
                    metr[key] = [wavg, True]
                    break
            nones = [v for v, w in list(metr.values()) if v == None]

    final_values = [list(metr.values()) for metr in [sales, profit, real_profit, interest, roe, per, pbr]]
    none_data_flag = []
   
    for value in final_values :
        if all(v == None for v, w in value) or len([v for v, w in value if v != None]) == 1 :
            none_data_flag.append(True)
        else :
            none_data_flag.append(False)
    
    if all(none_data_flag) :
        return f'''
            <!-- {txt} -->
            <div class="fs-head">
                <div class="fs-title">{txt}</div>
            </div>
            <div class="none-data">{txt} 데이터가 아직 없습니다.</div>
            <!-- {txt} END -->'''

    # 영업이익률
    profit_ratio = [round(((profit[key][0])/(value[0]))*100, 2) if all([profit[key][0] != None, value[0] != None]) else '' for key, value in sales.items()]
    # 순이익률
    real_profit_ratio = [round(((real_profit[key][0])/(value[0]))*100, 2) if all([real_profit[key][0] != None, value[0] != None]) else '' for key, value in sales.items()]

    html_fs = f'''
            <!-- {txt} -->
            <div class="fs-head">
                <div class="fs-title">{txt}</div>
                <div class="fs-unit">단위: 억원, %</div>
            </div>
            <div class="fs-div">
                <table class="fs-data">
                    <thead>
                        <th>IFRS(연결)</th>
                        <th>{table_head[0]}</th>
                        <th>{table_head[1]}</th>
                        <th>{table_head[2]}</th>
                        <th>{table_head[3]}</th>
                    </thead>
                    <tbody>
                        <tr>
                            <th>매출액</th>
                            <td{' class="weight-value"' if sales['c1'][1] else ''}><span{' class="bad-value"' if int_cond(sales['c1'][0]) and (sales['c1'][0] < 50) and (txt=='연간') else ''}>{comma(sales['c1'][0])}</span></td>
                            <td{' class="weight-value"' if sales['c2'][1] else ''}><span{' class="bad-value"' if int_cond(sales['c2'][0]) and (sales['c2'][0] < 50) and (txt=='연간') else ''}>{comma(sales['c2'][0])}</span></td>
                            <td{' class="weight-value"' if sales['c3'][1] else ''}><span{' class="bad-value"' if int_cond(sales['c3'][0]) and (sales['c3'][0] < 50) and (txt=='연간') else ''}>{comma(sales['c3'][0])}</span></td>
                            <td{' class="weight-value"' if sales['c4'][1] else ''}><span{' class="bad-value"' if int_cond(sales['c4'][0]) and (sales['c4'][0] < 50) and (txt=='연간') else ''}>{comma(sales['c4'][0])}</span></td>
                        </tr>
                        <tr>
                            <th>영업이익</th>
                            <td{' class="weight-value"' if profit['c1'][1] else ''}><span{' class="bad-value"' if int_cond(profit['c1'][0]) and (profit['c1'][0] < 0) else ''}>{comma(profit['c1'][0])}</span></td>
                            <td{' class="weight-value"' if profit['c2'][1] else ''}><span{' class="bad-value"' if int_cond(profit['c2'][0]) and (profit['c2'][0] < 0) else ''}>{comma(profit['c2'][0])}</span></td>
                            <td{' class="weight-value"' if profit['c3'][1] else ''}><span{' class="bad-value"' if int_cond(profit['c3'][0]) and (profit['c3'][0] < 0) else ''}>{comma(profit['c3'][0])}</span></td>
                            <td{' class="weight-value"' if profit['c4'][1] else ''}><span{' class="bad-value"' if int_cond(profit['c4'][0]) and (profit['c4'][0] < 0) else ''}>{comma(profit['c4'][0])}</span></td>
                        </tr>
                        <tr>
                            <th>영업이익률</th>
                            <td{' class="weight-value"' if profit['c1'][1] else ''}><span{(' class="good-value"' if profit_ratio[0] >= 15 else ' class="bad-value"' if profit_ratio[0] < 5 else '') if type(profit_ratio[0]) in [float, int] else ''}>{comma(profit_ratio[0])}</span></td>
                            <td{' class="weight-value"' if profit['c2'][1] else ''}><span{(' class="good-value"' if profit_ratio[1] >= 15 else ' class="bad-value"' if profit_ratio[1] < 5 else '') if type(profit_ratio[1]) in [float, int] else ''}>{comma(profit_ratio[1])}</span></td>
                            <td{' class="weight-value"' if profit['c3'][1] else ''}><span{(' class="good-value"' if profit_ratio[2] >= 15 else ' class="bad-value"' if profit_ratio[2] < 5 else '') if type(profit_ratio[2]) in [float, int] else ''}>{comma(profit_ratio[2])}</span></td>
                            <td{' class="weight-value"' if profit['c4'][1] else ''}><span{(' class="good-value"' if profit_ratio[3] >= 15 else ' class="bad-value"' if profit_ratio[3] < 5 else '') if type(profit_ratio[3]) in [float, int] else ''}>{comma(profit_ratio[3])}</span></td>
                        </tr>
                        <tr>
                            <th>당기순이익</th>
                            <td{' class="weight-value"' if real_profit['c1'][1] else ''}><span{' class="bad-value"' if int_cond(real_profit['c1'][0]) and (real_profit['c1'][0] < 0) else ''}>{comma(real_profit['c1'][0])}</span></td>
                            <td{' class="weight-value"' if real_profit['c2'][1] else ''}><span{' class="bad-value"' if int_cond(real_profit['c2'][0]) and (real_profit['c2'][0] < 0) else ''}>{comma(real_profit['c2'][0])}</span></td>
                            <td{' class="weight-value"' if real_profit['c3'][1] else ''}><span{' class="bad-value"' if int_cond(real_profit['c3'][0]) and (real_profit['c3'][0] < 0) else ''}>{comma(real_profit['c3'][0])}</span></td>
                            <td{' class="weight-value"' if real_profit['c4'][1] else ''}><span{' class="bad-value"' if int_cond(real_profit['c4'][0]) and (real_profit['c4'][0] < 0) else ''}>{comma(real_profit['c4'][0])}</span></td>
                        </tr>
                        <tr>
                            <th>당기순이익률</th>
                            <td{' class="weight-value"' if real_profit['c1'][1] else ''}><span{(' class="good-value"' if real_profit_ratio[0] >= 10 else ' class="bad-value"' if real_profit_ratio[0] < 3 else '') if real_profit_ratio[0] in [float, int] else ''}>{comma(real_profit_ratio[0])}</span></td>
                            <td{' class="weight-value"' if real_profit['c2'][1] else ''}><span{(' class="good-value"' if real_profit_ratio[1] >= 10 else ' class="bad-value"' if real_profit_ratio[1] < 3 else '') if real_profit_ratio[1] in [float, int] else ''}>{comma(real_profit_ratio[1])}</span></td>
                            <td{' class="weight-value"' if real_profit['c3'][1] else ''}><span{(' class="good-value"' if real_profit_ratio[2] >= 10 else ' class="bad-value"' if real_profit_ratio[2] < 3 else '') if real_profit_ratio[2] in [float, int] else ''}>{comma(real_profit_ratio[2])}</span></td>
                            <td{' class="weight-value"' if real_profit['c4'][1] else ''}><span{(' class="good-value"' if real_profit_ratio[3] >= 10 else ' class="bad-value"' if real_profit_ratio[3] < 3 else '') if real_profit_ratio[3] in [float, int] else ''}>{comma(real_profit_ratio[3])}</span></td>
                        </tr>
                        <tr>
                            <th>지배주주지분</th>
                            <td{' class="weight-value"' if interest['c1'][1] else ''}>{comma(interest['c1'][0])}</td>
                            <td{' class="weight-value"' if interest['c2'][1] else ''}>{comma(interest['c2'][0])}</td>
                            <td{' class="weight-value"' if interest['c3'][1] else ''}>{comma(interest['c3'][0])}</td>
                            <td{' class="weight-value"' if interest['c4'][1] else ''}>{comma(interest['c4'][0])}</td>
                        </tr>
                        <tr>
                            <th>ROE</th>
                            <td{' class="weight-value"' if roe['c1'][1] else ''}><span{roe_value(roe['c1'][0])}>{comma(roe['c1'][0], 2)}</span></td>
                            <td{' class="weight-value"' if roe['c2'][1] else ''}><span{roe_value(roe['c2'][0])}>{comma(roe['c2'][0], 2)}</span></td>
                            <td{' class="weight-value"' if roe['c3'][1] else ''}><span{roe_value(roe['c3'][0])}>{comma(roe['c3'][0], 2)}</span></td>
                            <td{' class="weight-value"' if roe['c4'][1] else ''}><span{roe_value(roe['c4'][0])}>{comma(roe['c4'][0], 2)}</span></td>
                        </tr>
                        <tr>
                            <th>PER</th>
                            <td{' class="weight-value"' if per['c1'][1] else ''}><span{per_value(per['c1'][0])}>{comma(per['c1'][0], 2)}</span></td>
                            <td{' class="weight-value"' if per['c2'][1] else ''}><span{per_value(per['c2'][0])}>{comma(per['c2'][0], 2)}</span></td>
                            <td{' class="weight-value"' if per['c3'][1] else ''}><span{per_value(per['c3'][0])}>{comma(per['c3'][0], 2)}</span></td>
                            <td{' class="weight-value"' if per['c4'][1] else ''}><span{per_value(per['c4'][0])}>{comma(per['c4'][0], 2)}</span></td>
                        </tr>
                        <tr>
                            <th>PBR</th>
                            <td{' class="weight-value"' if pbr['c1'][1] else ''}><span{pbr_value(pbr['c1'][0])}>{comma(pbr['c1'][0], 2)}</span></td>
                            <td{' class="weight-value"' if pbr['c2'][1] else ''}><span{pbr_value(pbr['c2'][0])}>{comma(pbr['c2'][0], 2)}</span></td>
                            <td{' class="weight-value"' if pbr['c3'][1] else ''}><span{pbr_value(pbr['c3'][0])}>{comma(pbr['c3'][0], 2)}</span></td>
                            <td{' class="weight-value"' if pbr['c4'][1] else ''}><span{pbr_value(pbr['c4'][0])}>{comma(pbr['c4'][0], 2)}</span></td>
                        </tr>
                    </tbody>
                </table>
            </div>
            <!-- {txt} END -->
            <!-- {txt} Chart -->
            <div class="fs-chard-div">
                <canvas class="{chart_class}"></canvas>
                <script>
                    financeSetChart('.{chart_class}', {table_head}, [
                        {[v for v, f in sales.values()]},
                        {profit_ratio},
                        {real_profit_ratio},
                    ]);
                </script>
            </div>
            <!-- {txt} Chart END -->
'''

    return html_fs

# HTML 재무제표(연간/분기)
html_fs_table = f'''
{fs_table(sfsy, '연간')}
{fs_table(sfsq, '분기')}
'''

with open('fs_test.html', 'w') as f :
    f.write(html_fs_table)


In [245]:
# HTML 재무상태표
deacha_head = [date for date, value in list(sdeacha.values())[0]]
deacha_value = {key:[val[1] for val in value] for key, value in sdeacha.items()}

# 부채 비율
deacha_ratio = [round((deacha_value['부채'][idx]/v)*100, 2) for idx, v in enumerate(deacha_value['자본'])]

step_size = '0'*len(str(max(deacha_value['자산'])))
step_size = int('1' + step_size[1:])

html_daecha = f'''
        <!-- 재무상태표 -->
        <div class="fs-head" style="margin-top: 20px">
            <div class="fs-title">재무상태표</div>
            <div class="fs-unit">단위: 억원, %</div>
        </div>
        <div class="fs-div">
            <table class="fs-data">
                <thead>
                    <th>IFRS(연결)</th>
                    <th>{deacha_head[0]}</th>
                    <th>{deacha_head[1]}</th>
                    <th>{deacha_head[2]}</th>
                    <th>{deacha_head[3]}</th>
                </thead>
                <tbody>
                    <tr>
                        <th>자산</th>
                        <td>{comma(deacha_value['자산'][0])}</td>
                        <td>{comma(deacha_value['자산'][1])}</td>
                        <td>{comma(deacha_value['자산'][2])}</td>
                        <td>{comma(deacha_value['자산'][3])}</td>
                    </tr>
                    <tr>
                        <th>부채</th>
                        <td>{comma(deacha_value['부채'][0])}</td>
                        <td>{comma(deacha_value['부채'][1])}</td>
                        <td>{comma(deacha_value['부채'][2])}</td>
                        <td>{comma(deacha_value['부채'][3])}</td>
                    </tr>
                    <tr>
                        <th>자본</th>
                        <td>{comma(deacha_value['자본'][0])}</td>
                        <td>{comma(deacha_value['자본'][1])}</td>
                        <td>{comma(deacha_value['자본'][2])}</td>
                        <td>{comma(deacha_value['자본'][3])}</td>
                    </tr>
                    <tr>
                        <th>부채비율</th>
                        <td><span{' class="good-value"' if deacha_ratio[0] <= 100 else ' class="bad-value"' if deacha_ratio[0] >= 150 else ''}>{comma(deacha_ratio[0])}</span></td>
                        <td><span{' class="good-value"' if deacha_ratio[1] <= 100 else ' class="bad-value"' if deacha_ratio[1] >= 150 else ''}>{comma(deacha_ratio[1])}</span></td>
                        <td><span{' class="good-value"' if deacha_ratio[2] <= 100 else ' class="bad-value"' if deacha_ratio[2] >= 150 else ''}>{comma(deacha_ratio[2])}</span></td>
                        <td><span{' class="good-value"' if deacha_ratio[3] <= 100 else ' class="bad-value"' if deacha_ratio[3] >= 150 else ''}>{comma(deacha_ratio[3])}</span></td>
                    </tr>
                </tbody>
            </table>
        </div>
        <!-- 재무상태표 END -->
        <!-- 재무상태표 Chart -->
        <div class="fs-chard-div">
            <canvas class="fs-bar-chart"></canvas>
            <script>
                const fsBarChart = document.querySelector('.fs-bar-chart').getContext('2d');
                const deptRatio = {deacha_ratio};
                const fsBarCanvas = new Chart(fsBarChart, {{
                    type: 'bar',
                    data: {{
                        labels: ['2020/12', '2021/12', '2022/12', '2023/03'],
                        datasets: [{{
                                label: '자본',
                                data: {deacha_value['자본']},
                                backgroundColor: 'rgba(16, 163, 127, 0.2)',
                                borderColor: 'rgba(16, 163, 127, 1)',
                                borderWidth: 1,
                                stack: 'combined',
                                yAxisID: 'y1',
                            }},
                            {{
                                label: '부채',
                                data: {deacha_value['부채']},
                                backgroundColor: 'rgba(255, 99, 132, 0.2)',
                                borderColor: 'rgba(255, 99, 132, 1)',
                                borderWidth: 1,
                                stack: 'combined',
                                yAxisID: 'y1',
                            }}, {{
                                label: '부채비율',
                                data: deptRatio,
                                backgroundColor: 'rgba(0, 0, 128, 0.2)',
                                borderColor: 'rgba(0, 0, 128, 1)',
                                borderWidth: 1,
                                type: 'line',
                                yAxisID: 'y2',
                            }}
                        ]
                    }},
                    options: {{
                        scales: {{
                            y1: {{
                                beginAtZero: true,
                                stacked: true,
                                position: 'left',
                                ticks: {{
                                    stepSize: {step_size},
                                    callback: function(value, index, ticks) {{
                                        return `${{value.toLocaleString()}}억원`;
                                    }}
                                }}
                            }},
                            y2: {{
                                beginAtZero: true,
                                position: 'right',
                                grid: {{
                                    drawOnChartArea: false,
                                }},
                                ticks: {{
                                    callback: function(value, index, ticks) {{
                                        return `${{value.toLocaleString()}}%`;
                                    }}
                                }},
                                min: Math.round((Math.min(...deptRatio) / 5) * 5) - 5,
                                max: Math.round((Math.max(...deptRatio) / 5) * 5) + 5,
                                stepSize: 5
                            }}
                        }},
                    }}
                }});
            </script>
        </div>
        <!-- 재무상태표 Chart END -->
'''
with open('deacha.html', 'w') as f :
    f.write(html_daecha)

In [246]:
# HTML 재무비율
rate_head = [date for date, value in list(srate.values())[0]]
rate_value = {key:[val[1] for val in value] for key, value in srate.items()}

html_fs_ratio = f'''
        <!-- 재무비율 -->
        <div class="fs-head" style="margin-top: 20px">
            <div class="fs-title">재무비율</div>
            <div class="fs-unit">단위: %, 억원</div>
        </div>
        <div class="fs-div">
            <table class="fs-data">
                <thead>
                    <th>IFRS(연결)</th>
                    <th>{rate_head[0]}</th>
                    <th>{rate_head[1]}</th>
                    <th>{rate_head[2]}</th>
                    <th>{rate_head[3]}</th>
                </thead>
                <tbody>
                    <tr>
                        <th>유동비율</th>
                        <td><span{' class="good-value"' if rate_value['유동비율'][0] >= 200 else ' class="bad-value"' if rate_value['유동비율'][0] < 100 else ''}>{comma(rate_value['유동비율'][0])}</span></td>
                        <td><span{' class="good-value"' if rate_value['유동비율'][1] >= 200 else ' class="bad-value"' if rate_value['유동비율'][1] < 100 else ''}>{comma(rate_value['유동비율'][1])}</span></td>
                        <td><span{' class="good-value"' if rate_value['유동비율'][2] >= 200 else ' class="bad-value"' if rate_value['유동비율'][2] < 100 else ''}>{comma(rate_value['유동비율'][2])}</span></td>
                        <td><span{' class="good-value"' if rate_value['유동비율'][3] >= 200 else ' class="bad-value"' if rate_value['유동비율'][3] < 100 else ''}>{comma(rate_value['유동비율'][3])}</span></td>
                    </tr>
                    <tr>
                        <th>당좌비율</th>
                        <td><span{' class="good-value"' if rate_value['당좌비율'][0] >= 150 else ' class="bad-value"' if rate_value['당좌비율'][0] < 100 else ''}>{comma(rate_value['당좌비율'][0])}</span></td>
                        <td><span{' class="good-value"' if rate_value['당좌비율'][1] >= 150 else ' class="bad-value"' if rate_value['당좌비율'][1] < 100 else ''}>{comma(rate_value['당좌비율'][1])}</span></td>
                        <td><span{' class="good-value"' if rate_value['당좌비율'][2] >= 150 else ' class="bad-value"' if rate_value['당좌비율'][2] < 100 else ''}>{comma(rate_value['당좌비율'][2])}</span></td>
                        <td><span{' class="good-value"' if rate_value['당좌비율'][3] >= 150 else ' class="bad-value"' if rate_value['당좌비율'][3] < 100 else ''}>{comma(rate_value['당좌비율'][3])}</span></td>
                    </tr>
                    <tr>
                        <th>부채비율</th>
                        <td><span{' class="good-value"' if rate_value['부채비율'][0] < 150 else ' class="bad-value"' if rate_value['부채비율'][0] >= 200 else ''}>{comma(rate_value['부채비율'][0])}</span></td>
                        <td><span{' class="good-value"' if rate_value['부채비율'][1] < 150 else ' class="bad-value"' if rate_value['부채비율'][1] >= 200 else ''}>{comma(rate_value['부채비율'][1])}</span></td>
                        <td><span{' class="good-value"' if rate_value['부채비율'][2] < 150 else ' class="bad-value"' if rate_value['부채비율'][2] >= 200 else ''}>{comma(rate_value['부채비율'][2])}</span></td>
                        <td><span{' class="good-value"' if rate_value['부채비율'][3] < 150 else ' class="bad-value"' if rate_value['부채비율'][3] >= 200 else ''}>{comma(rate_value['부채비율'][3])}</span></td>
                    </tr>
                    <tr>
                        <th>유보율</th>
                        <td>{comma(rate_value['유보율'][0])}</td>
                        <td>{comma(rate_value['유보율'][1])}</td>
                        <td>{comma(rate_value['유보율'][2])}</td>
                        <td>{comma(rate_value['유보율'][3])}</td>
                    </tr>
                </tbody>
            </table>
        </div>
        <div class="fs-comment-text">유보율은 높을수록 좋다.</div>
        <!-- 재무비율 END -->'''

with open('ratio.html', 'w') as f :
    f.write(html_fs_ratio)

In [247]:
# HTML 현금흐름표
cash_head = [date for date, value in list(scash.values())[0]]
cash_value = {key:[val[1] for val in value] for key, value in scash.items()}

# 영업활동
sales_act = cash_value['영업활동으로인한현금흐름']
# 투자활동
invest_act = cash_value['투자활동으로인한현금흐름']
# 재무활동
finance_act = cash_value['재무활동으로인한현금흐름']

# 구분 조합
sales_pm = ['+' if i > 0 else '-' for i in sales_act]
invest_pm = ['+' if i > 0 else '-' for i in invest_act]
finance_pm = ['+' if i > 0 else '-' for i in finance_act]

pms = zip(sales_pm, invest_pm, finance_pm)
pms_tx = ['/'.join(i) for i in list(pms)]

# 구분 good / bad 
def cash_gubun(comb) :
    if comb in ['+/-/-', '+/-/+'] :
        return f'<span class="good-value">{comb}</span>'
    
    if comb in ['-/+/-', '-/+/+'] :
        return f'<span class="bad-value">{comb}</span>'
    
    return comb

cash_comb_name = {
    '+/-/-' : '<span class="good-value">우량</span>',
    '+/-/+' : '<span class="good-value">성장</span>',
    '+/+/-' : '과도기',
    '-/-/-' : '재활',
    '-/-/+' : '재활',
    '-/+/-' : '<span class="bad-value">위험</span>',
    '-/+/+' : '<span class="bad-value">접근금지</span>',
}


html_cash = f'''
        <!-- 현금흐름표 -->
        <div class="fs-head" style="margin-top: 20px">
            <div class="fs-title">현금흐름표</div>
            <div class="fs-unit">단위: 억원</div>
        </div>
        <div class="fs-div">
            <table class="fs-data">
                <thead>
                    <th>IFRS(연결)</th>
                    <th>{cash_head[0]}</th>
                    <th>{cash_head[1]}</th>
                    <th>{cash_head[2]}</th>
                    <th>{cash_head[3]}</th>
                </thead>
                <tbody>
                    <tr>
                        <th>영업활동</th>
                        <td><span{' class="minus-value"' if sales_act[0] < 0 else ''}>{comma(sales_act[0])}</span></td>
                        <td><span{' class="minus-value"' if sales_act[1] < 0 else ''}>{comma(sales_act[1])}</span></td>
                        <td><span{' class="minus-value"' if sales_act[2] < 0 else ''}>{comma(sales_act[2])}</span></td>
                        <td><span{' class="minus-value"' if sales_act[3] < 0 else ''}>{comma(sales_act[3])}</span></td>
                    </tr>
                    <tr>
                        <th>투자활동</th>
                        <td><span{' class="minus-value"' if invest_act[0] < 0 else ''}>{comma(invest_act[0])}</span></td>
                        <td><span{' class="minus-value"' if invest_act[1] < 0 else ''}>{comma(invest_act[1])}</span></td>
                        <td><span{' class="minus-value"' if invest_act[2] < 0 else ''}>{comma(invest_act[2])}</span></td>
                        <td><span{' class="minus-value"' if invest_act[3] < 0 else ''}>{comma(invest_act[3])}</span></td>
                    </tr>
                    <tr>
                        <th>재무활동</th>
                        <td><span{' class="minus-value"' if finance_act[0] < 0 else ''}>{comma(finance_act[0])}</span></td>
                        <td><span{' class="minus-value"' if finance_act[1] < 0 else ''}>{comma(finance_act[1])}</span></td>
                        <td><span{' class="minus-value"' if finance_act[2] < 0 else ''}>{comma(finance_act[2])}</span></td>
                        <td><span{' class="minus-value"' if finance_act[3] < 0 else ''}>{comma(finance_act[3])}</span></td>
                    </tr>
                    <tr>
                        <th rowspan="2">구분</th>
                        <td>{cash_gubun(pms_tx[0])}</td>
                        <td>{cash_gubun(pms_tx[1])}</td>
                        <td>{cash_gubun(pms_tx[2])}</td>
                        <td>{cash_gubun(pms_tx[3])}</td>
                    </tr>
                    <tr>
                        <td>{cash_comb_name[pms_tx[0]]}</td>
                        <td>{cash_comb_name[pms_tx[1]]}</td>
                        <td>{cash_comb_name[pms_tx[2]]}</td>
                        <td>{cash_comb_name[pms_tx[3]]}</td>
                    </tr>
                </tbody>
            </table>
        </div>
        <!-- 현금흐름표 END -->
        <div class="a-line"></div>'''

with open('cash.html', 'w') as f :
    f.write(html_cash)

In [248]:
# 여기서부터...
html = '''
        <!-- 적정주가 계산 파트 -->
        <div class="report-title">적정주가 계산</div>
        <!-- 영업이익*PER 적정 주가 계산 -->
        <div class="fs-head" style="margin-top: 20px">
            <div class="fs-title">영업이익*PER</div>
        </div>
        <div class="standard-info">
            <div class="info">
                <div class="info-cell">
                    <div class="cell-head">영업이익</div>
                    <div class="cell-desc">14,160억원</div>
                </div>

                <div class="info-cell">
                    <div class="cell-head">기준</div>
                    <div class="cell-desc">2023/12(E)</div>
                </div>
            </div>
            <div class="info-one">
                <div class="cell-head">현재주가</div>
                <div class="cell-desc">318,500원</div>
            </div>
        </div>
        <div class="fs-div" style="margin-top: 10px">
            <table class="fs-data">
                <thead>
                    <th>구분</th>
                    <th>PER (%)</th>
                    <th>기업가치</th>
                    <th>적정주가</th>
                </thead>
                <tbody>
                    <tr>
                        <th>올해 예상</th>
                        <td>11.32</td>
                        <td>160,291.2억원</td>
                        <td>1,064,762원</td>
                    </tr>
                    <tr>
                        <th>업종 평균</th>
                        <td>11.32</td>
                        <td>160,291.2억원</td>
                        <td>1,064,762원</td>
                    </tr>
                    <tr>
                        <th>별도 조정</th>
                        <td>
                            <input class="custom-input per-input" value="10" type="number" min="0" max="100" step="1" />
                        </td>
                        <td><span class="custom-value per-fair-value"></span></td>
                        <td><span class="custom-value per-fair-price"></span></td>
                    </tr>
                </tbody>
            </table>
        </div>
        <!-- 영업이익*PER END -->
        <!-- 영업이익*PER Chart -->
        <div class="fs-chard-div">
            <canvas class="fs-per-chart"></canvas>
            <script>
                const perCanvas = document.querySelector('.fs-per-chart').getContext('2d');
                const nowValue = 318500;
                let calcPer = 0;
                const perChart = new Chart(perCanvas, {
                    type: 'line',
                    data: {
                        labels: ['올해 예상', '업종 평균', '별도 조정'],
                        datasets: [{
                            label: '적정주가',
                            data: [1064762, 1064762, calcPer],
                            backgroundColor: 'rgba(16, 163, 127, 0.2)',
                            borderColor: 'rgba(16, 163, 127, 1)',
                            borderWidth: 1,
                            type: 'bar'
                        }]
                    },
                    options: {
                        scales: {
                            y: {
                                beginAtZero: true,
                                position: 'left',
                                ticks: {
                                    callback: function(value, index, ticks) {
                                        return `${value.toLocaleString()}원`;
                                    }
                                }
                            }
                        },
                        plugins: {
                            legend: {
                                onClick: (e) => e.stopPropagation()
                            },
                            annotation: {
                                annotations: {
                                    currentValue: {
                                        type: 'line',
                                        mode: 'horizontal',
                                        scaleID: 'y',
                                        value: nowValue,
                                        borderColor: 'rgba(255, 127, 80, 1)',
                                        borderWidth: 1.5,
                                        label: {
                                            content: '현재 주가',
                                            enabled: true,
                                            position: 'top',
                                        }
                                    },
                                    label1: {
                                        type: 'label',
                                        yValue: nowValue,
                                        content: ['현재 주가', `${nowValue.toLocaleString()}원`],
                                        color: 'rgba(255, 127, 80, 1)',
                                        font: {
                                            size: 13,
                                        }
                                    }
                                }
                            }
                        }
                    }
                });

                const perInput = document.querySelector('.per-input');
                const perHandler = (perValue) => {
                    const fairValue = document.querySelector('.per-fair-value');
                    const fairPrice = document.querySelector('.per-fair-price');

                    if (parseInt(perValue) > 100) {
                        perValue = 100;
                    }

                    const earning = 14160; // 영업이익
                    const stockCnt = 1504186; // 발행주식수
                    const per = parseFloat(perValue);

                    // 기업가치
                    let calcEarning = earning * per;
                    let calcPrice = (calcEarning / stockCnt) * 10000000;

                    calcEarning = calcEarning.toFixed(1);
                    calcPrice = calcPrice.toFixed(0);

                    if (isNaN(calcEarning)) {
                        fairValue.innerHTML = '';
                        fairPrice.innerHTML = '';
                        calcPrice = 0;
                    } else {
                        fairValue.innerHTML = `${parseFloat(calcEarning).toLocaleString()}억원`;
                        fairPrice.innerHTML = `${parseInt(calcPrice).toLocaleString()}원`;
                    }

                    calcPer = calcPrice;
                    perChart.data.datasets[0].data[2] = calcPer;

                    perChart.update();
                };

                perInput.addEventListener('input', (event) => {
                    perHandler(event.target.value);
                });
                perHandler(10);
            </script>
        </div>
        <!-- 영업이익*PER Chart END -->
        <!-- SRIM 계산 -->
        <div class="fs-head" style="margin-top: 20px">
            <div class="fs-title">SRIM</div>
        </div>
        <div class="standard-info">
            <div class="info-one">
                <div class="cell-head">발행주식수</div>
                <div class="cell-desc">402,044,203</div>
            </div>
            <div class="info-one">
                <div class="cell-head">자사주</div>
                <div class="cell-desc">6,962,511</div>
            </div>

            <div class="info" style="margin-top: 15px;">
                <div class="info-cell">
                    <div class="cell-head">지배주주지분</div>
                    <div class="cell-desc">393,379</div>
                </div>

                <div class="info-cell">
                    <div class="cell-head">BBB-</div>
                    <div class="cell-desc">10.85%</div>
                </div>
                <div class="info-one">
                    <div class="cell-head">예상 ROE</div>
                    <div class="cell-desc">19.33%</div>
                </div>
                <div class="info-one">
                    <div class="cell-head">가중 ROE</div>
                    <div class="cell-desc">13.02%</div>
                </div>
            </div>
        </div>
        <div class="srim-page">
            <div class="srim-roe-head">
                <div class="srim-roe roe-selected">예상 ROE</div>
                <div class="srim-roe">가중 ROE</div>
                <div class="srim-roe-info">기준 ROE 선택</div>
            </div>
            <div class="fs-div srim-result" style="margin-top: 10px">
                <table class="fs-data srim-table">
                    <thead>
                        <th>초과이익</th>
                        <th>기업가치</th>
                        <th>적정주가</th>
                    </thead>
                    <tbody>
                        <tr>
                            <th>이익 지속</th>
                            <td>1700,830.97억</td>
                            <td>177,389원</td>
                        </tr>
                        <tr>
                            <th>10% 감소</th>
                            <td>537,372.69억</td>
                            <td>136,016원</td>
                        </tr>
                        <tr>
                            <th>20% 감소</th>
                            <td>479,884.13억</td>
                            <td>121,465원</td>
                        </tr>
                        <tr>
                            <th>30% 감소</th>
                            <td>379,884.13억</td>
                            <td>111,465원</td>
                        </tr>
                        <tr>
                            <th>40% 감소</th>
                            <td>279,884.13억</td>
                            <td>91,465원</td>
                        </tr>
                        <tr>
                            <th>50% 감소</th>
                            <td>179,884.13억</td>
                            <td>71,465원</td>
                        </tr>
                    </tbody>
                </table>

                <table class="fs-data srim-table post-fs-hidden">
                    <thead>
                        <th>초과이익</th>
                        <th>기업가치</th>
                        <th>적정주가</th>
                    </thead>
                    <tbody>
                        <tr>
                            <th>이익 지속</th>
                            <td>1800,830.97억</td>
                            <td>187,389원</td>
                        </tr>
                        <tr>
                            <th>10% 감소</th>
                            <td>547,372.69억</td>
                            <td>146,016원</td>
                        </tr>
                        <tr>
                            <th>20% 감소</th>
                            <td>479,884.13억</td>
                            <td>131,465원</td>
                        </tr>
                        <tr>
                            <th>30% 감소</th>
                            <td>379,884.13억</td>
                            <td>111,465원</td>
                        </tr>
                        <tr>
                            <th>40% 감소</th>
                            <td>279,884.13억</td>
                            <td>91,465원</td>
                        </tr>
                        <tr>
                            <th>50% 감소</th>
                            <td>179,884.13억</td>
                            <td>71,465원</td>
                        </tr>
                    </tbody>
                </table>
            </div>
        </div>
        <!-- SRIM END -->
        <!-- SRIM Chart -->
        <div class="fs-chard-div">
            <canvas class="fs-srim-chart"></canvas>
        </div>
        <script>
            // SRIM CHART
            const srimCanvas = document.querySelector('.fs-srim-chart').getContext('2d');
            const srimResults = [
                [267613, 369561, 405503, 425503, 455503, 485503],
                [485503, 455503, 425503, 405503, 369561, 267613]
            ]
            let setSRIM = srimResults[0];
            const srimChart = new Chart(srimCanvas, {
                type: 'line',
                data: {
                    labels: ['이익 지속', '10% 감소', '20% 감소', '30% 감소', '40% 감소', '50% 감소'],
                    datasets: [{
                        label: '적정주가',
                        data: setSRIM,
                        backgroundColor: 'rgba(16, 163, 127, 0.2)',
                        borderColor: 'rgba(16, 163, 127, 1)',
                        borderWidth: 1,
                        type: 'bar'
                    }]
                },
                options: {
                    scales: {
                        y: {
                            beginAtZero: true,
                            position: 'left',
                            ticks: {
                                callback: function(value, index, ticks) {
                                    return `${value.toLocaleString()}원`;
                                }
                            }
                        }
                    },
                    plugins: {
                        legend: {
                            onClick: (e) => e.stopPropagation()
                        },
                        annotation: {
                            annotations: {
                                currentValue: {
                                    type: 'line',
                                    mode: 'horizontal',
                                    scaleID: 'y',
                                    value: nowValue,
                                    borderColor: 'rgba(255, 127, 80, 1)',
                                    borderWidth: 1.5,
                                    label: {
                                        content: '현재 주가',
                                        enabled: true,
                                        position: 'top',
                                    }
                                },
                                label1: {
                                    type: 'label',
                                    yValue: nowValue,
                                    content: ['현재 주가', `${nowValue.toLocaleString()}원`],
                                    color: 'rgba(255, 127, 80, 1)',
                                    font: {
                                        size: 13,
                                    }
                                }
                            }
                        }
                    }
                }
            });
            // ROE 선택 기능
            const roeBtn = document.querySelectorAll('.srim-roe');
            const roeSelected = 'roe-selected';
            const roeHandler = (index) => {
                const roeHead = document.querySelector('.srim-roe-head');
                const selectedRoe = roeHead.querySelector(`.${roeSelected}`);
                const roeTables = document.querySelectorAll('.srim-table');
                selectedRoe.classList.remove(roeSelected);

                roeBtn[index].classList.add(roeSelected);
                roeTables.forEach((table) => {
                    table.classList.add('post-fs-hidden');
                });
                roeTables[index].classList.remove('post-fs-hidden');

                // Chart update
                setSRIM = srimResults[index];
                srimChart.data.datasets[0].data = setSRIM;

                srimChart.update();
            };

            roeBtn.forEach((roe, index) => {
                roe.addEventListener('click', () => {
                    roeHandler(index);
                });
            });
        </script>
    </div>
    <!-- SRIM Chart END -->
    <div class="a-line"></div>
'''

In [249]:
# HTML Body (END)
html_fs_body = f'''
{html_head}
{html_info}
<div class="report-title">재무정보</div>
<div class="financial-statements">
    <div class="fs-table">
{html_fs_table}
{html_daecha}
{html_fs_ratio}
{html_cash}
    </div>
</div>
<!-- END -->
'''

with open('final.html', 'w') as f :
    f.write(html_fs_body)


In [250]:
# # backup 
# html = '''
# <!-- 재무제표 -->
# <div class="report-title">재무정보</div>
# <div class="financial-statements">
#     <div class="fs-table">
#         <!-- 연간 -->
#         <div class="fs-head">
#             <div class="fs-title">연간</div>
#             <div class="fs-unit">단위: 억원, %</div>
#         </div>
#         <div class="fs-div">
#             <table class="fs-data">
#                 <thead>
#                     <th>IFRS(연결)</th>
#                     <th>2020/12</th>
#                     <th>2021/12</th>
#                     <th>2022/12</th>
#                     <th>2023/12(E)</th>
#                 </thead>
#                 <tbody>
#                     <tr>
#                         <th>매출액</th>
#                         <td>242,457</td>
#                         <td>262,892</td>
#                         <td>300,795</td>
#                         <td>307,855</td>
#                     </tr>
#                     <tr>
#                         <th>영업이익</th>
#                         <td>13596</td>
#                         <td>15244</td>
#                         <td>16647</td>
#                         <td>14160</td>
#                     </tr>
#                     <tr>
#                         <th>당기순이익</th>
#                         <td>7864</td>
#                         <td>8924</td>
#                         <td>8027</td>
#                         <td>6353</td>
#                     </tr>
#                     <tr>
#                         <th>지배주주지분</th>
#                         <td>52744</td>
#                         <td>61135</td>
#                         <td>67557</td>
#                         <td>71805</td>
#                     </tr>
#                     <tr>
#                         <th>ROE</th>
#                         <td class="good-value">13.5</td>
#                         <td class="good-value">10.76</td>
#                         <td>9.26</td>
#                         <td>6.55</td>
#                     </tr>
#                     <tr>
#                         <th>PER</th>
#                         <td class="good-value">9.1</td>
#                         <td>10.36</td>
#                         <td>10.46</td>
#                         <td>11.32</td>
#                     </tr>
#                     <tr>
#                         <th>PBR</th>
#                         <td>1.16</td>
#                         <td>1.02</td>
#                         <td>0.91</td>
#                         <td>0.71</td>
#                     </tr>
#                 </tbody>
#             </table>
#         </div>
#         <!-- 연간 END -->
#         <!-- 연간 Chart -->
#         <div class="fs-chard-div">
#             <canvas class="fs-y-chart"></canvas>
#             <script>
#                 // 연간/분기 차트 생성
#                 const financeSetChart = (className, labels, datas) => {
#                     const ctx = document.querySelector(className).getContext('2d');
#                     const myChart = new Chart(ctx, {
#                         type: 'line',
#                         data: {
#                             labels: labels,
#                             datasets: [{
#                                 label: '매출액',
#                                 data: datas[0],
#                                 backgroundColor: 'rgba(16, 163, 127, 0.2)',
#                                 borderColor: 'rgba(16, 163, 127, 1)',
#                                 borderWidth: 1,
#                                 yAxisID: 'y1',
#                                 type: 'bar'
#                             }, {
#                                 label: '영업이익률',
#                                 data: datas[1],
#                                 backgroundColor: 'rgba(255, 99, 132, 0.2)',
#                                 borderColor: 'rgba(255, 99, 132, 1)',
#                                 borderWidth: 1,
#                                 yAxisID: 'y2',
#                             }, {
#                                 label: '순이익률',
#                                 data: datas[2],
#                                 backgroundColor: 'rgba(0, 0, 128, 0.2)',
#                                 borderColor: 'rgba(0, 0, 128, 1)',
#                                 borderWidth: 1,
#                                 yAxisID: 'y2',
#                             }]
#                         },
#                         options: {
#                             scales: {
#                                 y1: {
#                                     position: 'left',
#                                     ticks: {
#                                         callback: function(value, index, ticks) {
#                                             return `${value.toLocaleString()}억원`;
#                                         }
#                                     }
#                                 },
#                                 y2: {
#                                     beginAtZero: true,
#                                     position: 'right',
#                                     grid: {
#                                         drawOnChartArea: false,
#                                     },
#                                     ticks: {
#                                         callback: function(value, index, ticks) {
#                                             return `${value.toLocaleString()}%`;
#                                         }
#                                     }
#                                 }
#                             }
#                         }
#                     });
#                 }

#                 financeSetChart('.fs-y-chart', ['2020/12', '2021/12', '2022/12', '2023/12(E)'], [
#                     [242457, 262892, 300795, 307855],
#                     [5.6, 5.8, 5.5, 4.6],
#                     [3.24, 3.4, 2.7, 2.1]
#                 ]);
#             </script>
#         </div>
#         <!-- 연간 Chart END -->
#         <!-- 분기 -->
#         <div class="fs-head" style="margin-top: 20px">
#             <div class="fs-title">분기</div>
#             <div class="fs-unit">단위: 억원, %</div>
#         </div>
#         <div class="fs-div">
#             <table class="fs-data">
#                 <thead>
#                     <th>IFRS(연결)</th>
#                     <th>2023/03</th>
#                     <th>2023/06(E)</th>
#                     <th>2023/09(E)</th>
#                     <th>2023/12(E)</th>
#                 </thead>
#                 <tbody>
#                     <tr>
#                         <th>매출액</th>
#                         <td>242,457</td>
#                         <td>262,892</td>
#                         <td>300,795</td>
#                         <td>307,855</td>
#                     </tr>
#                     <tr>
#                         <th>영업이익</th>
#                         <td>13596</td>
#                         <td>15244</td>
#                         <td>16647</td>
#                         <td>14160</td>
#                     </tr>
#                     <tr>
#                         <th>당기순이익</th>
#                         <td>7864</td>
#                         <td>8924</td>
#                         <td>8027</td>
#                         <td>6353</td>
#                     </tr>
#                     <tr>
#                         <th>ROE</th>
#                         <td>13.5</td>
#                         <td>10.76</td>
#                         <td>9.26</td>
#                         <td>6.55</td>
#                     </tr>
#                     <tr>
#                         <th>PER</th>
#                         <td>9.1</td>
#                         <td>10.36</td>
#                         <td>10.46</td>
#                         <td>11.32</td>
#                     </tr>
#                     <tr>
#                         <th>PBR</th>
#                         <td>1.16</td>
#                         <td>1.02</td>
#                         <td>0.91</td>
#                         <td>0.71</td>
#                     </tr>
#                 </tbody>
#             </table>
#         </div>
#         <!-- 분기 END -->
#         <!-- 분기 Chart -->
#         <div class="fs-chard-div">
#             <canvas class="fs-q-chart"></canvas>
#             <script>
#                 financeSetChart('.fs-q-chart', ['2023/03(E)', '2023/06(E)', '2022/09(E)', '2023/12(E)'], [
#                     [242457, 262892, 300795, 307855],
#                     [5.6, 5.8, 5.5, 4.6],
#                     [3.24, 3.4, 2.7, 2.1]
#                 ]);
#             </script>
#         </div>
#         <!-- 연간 Chart END -->
#         <!-- 재무상태표 -->
#         <div class="fs-head" style="margin-top: 20px">
#             <div class="fs-title">재무상태표</div>
#             <div class="fs-unit">단위: 억원</div>
#         </div>
#         <div class="fs-div">
#             <table class="fs-data">
#                 <thead>
#                     <th>IFRS(연결)</th>
#                     <th>2020/12</th>
#                     <th>2021/12</th>
#                     <th>2022/12</th>
#                     <th>2023/03</th>
#                 </thead>
#                 <tbody>
#                     <tr>
#                         <th>자산</th>
#                         <td>3,782,357</td>
#                         <td>4,266,212</td>
#                         <td>4,484,245</td>
#                         <td>4,540,918</td>
#                     </tr>
#                     <tr>
#                         <th>부채</th>
#                         <td>1,022,877</td>
#                         <td>1,217,212</td>
#                         <td>936,749</td>
#                         <td>942,924</td>
#                     </tr>
#                     <tr>
#                         <th>자본</th>
#                         <td>2,759,480</td>
#                         <td>3,048,999</td>
#                         <td>3,547,496</td>
#                         <td>3,597,994</td>
#                     </tr>
#                 </tbody>
#             </table>
#         </div>
#         <!-- 재무상태표 END -->
#         <!-- 재무상태표 Chart -->
#         <div class="fs-chard-div">
#             <canvas class="fs-bar-chart"></canvas>
#             <script>
#                 const fsBarChart = document.querySelector('.fs-bar-chart').getContext('2d');
#                 const deptRatio = [37.1, 39.9, 26.4, 26.2];
#                 const fsBarCanvas = new Chart(fsBarChart, {
#                     type: 'bar',
#                     data: {
#                         labels: ['2020/12', '2021/12', '2022/12', '2023/03'],
#                         datasets: [{
#                                 label: '자본',
#                                 data: [2759480, 3048999, 3547496, 3597994],
#                                 backgroundColor: 'rgba(16, 163, 127, 0.2)',
#                                 borderColor: 'rgba(16, 163, 127, 1)',
#                                 borderWidth: 1,
#                                 stack: 'combined',
#                                 yAxisID: 'y1',
#                             },
#                             {
#                                 label: '부채',
#                                 data: [1022877, 1217212, 936749, 942924],
#                                 backgroundColor: 'rgba(255, 99, 132, 0.2)',
#                                 borderColor: 'rgba(255, 99, 132, 1)',
#                                 borderWidth: 1,
#                                 stack: 'combined',
#                                 yAxisID: 'y1',
#                             }, {
#                                 label: '부채비율',
#                                 data: deptRatio,
#                                 backgroundColor: 'rgba(0, 0, 128, 0.2)',
#                                 borderColor: 'rgba(0, 0, 128, 1)',
#                                 borderWidth: 1,
#                                 type: 'line',
#                                 yAxisID: 'y2',
#                             }
#                         ]
#                     },
#                     options: {
#                         scales: {
#                             y1: {
#                                 beginAtZero: true,
#                                 stacked: true,
#                                 position: 'left',
#                                 ticks: {
#                                     stepSize: 1000000,
#                                     callback: function(value, index, ticks) {
#                                         return `${value.toLocaleString()}억원`;
#                                     }
#                                 }
#                             },
#                             y2: {
#                                 beginAtZero: true,
#                                 position: 'right',
#                                 grid: {
#                                     drawOnChartArea: false,
#                                 },
#                                 ticks: {
#                                     callback: function(value, index, ticks) {
#                                         return `${value.toLocaleString()}%`;
#                                     }
#                                 },
#                                 min: Math.round((Math.min(...deptRatio) / 5) * 5) - 5,
#                                 max: Math.round((Math.max(...deptRatio) / 5) * 5) + 5,
#                                 stepSize: 5
#                             }
#                         },
#                     }
#                 });
#             </script>
#         </div>
#         <!-- 재무상태표 Chart END -->

#         <!-- 재무비율 -->
#         <div class="fs-head" style="margin-top: 20px">
#             <div class="fs-title">재무비율</div>
#             <div class="fs-unit">단위: %, 억원</div>
#         </div>
#         <div class="fs-div">
#             <table class="fs-data">
#                 <thead>
#                     <th>IFRS(연결)</th>
#                     <th>2020/12</th>
#                     <th>2021/12</th>
#                     <th>2022/12</th>
#                     <th>2023/03</th>
#                 </thead>
#                 <tbody>
#                     <tr>
#                         <th>유동비율</th>
#                         <td>262.2</td>
#                         <td>247.6</td>
#                         <td>278.9</td>
#                         <td>282.0</td>
#                     </tr>
#                     <tr>
#                         <th>당좌비율</th>
#                         <td>219.8</td>
#                         <td>200.6</td>
#                         <td>212.2</td>
#                         <td>210.4</td>
#                     </tr>
#                     <tr>
#                         <th>부채비율</th>
#                         <td>37.1</td>
#                         <td>39.9</td>
#                         <td>26.4</td>
#                         <td>26.2</td>
#                     </tr>
#                     <tr>
#                         <th>유보율</th>
#                         <td>29,723.5</td>
#                         <td>32,906.5</td>
#                         <td>38,360.3</td>
#                         <td>38,898.8</td>
#                     </tr>
#                 </tbody>
#             </table>
#         </div>
#         <!-- 재무비율 END -->

#         <!-- 현금흐름표 -->
#         <div class="fs-head" style="margin-top: 20px">
#             <div class="fs-title">현금흐름표</div>
#             <div class="fs-unit">단위: 억원</div>
#         </div>
#         <div class="fs-div">
#             <table class="fs-data">
#                 <thead>
#                     <th>IFRS(연결)</th>
#                     <th>2020/12</th>
#                     <th>2021/12</th>
#                     <th>2022/12</th>
#                     <th>2023/03</th>
#                 </thead>
#                 <tbody>
#                     <tr>
#                         <th>영업활동</th>
#                         <td>20,018</td>
#                         <td>16,864</td>
#                         <td>16,273</td>
#                         <td>104</td>
#                     </tr>
#                     <tr>
#                         <th>투자활동</th>
#                         <td class="bad-value">-3,451</td>
#                         <td class="bad-value">-6,254</td>
#                         <td class="bad-value">-14,985</td>
#                         <td class="bad-value">-2,818</td>
#                     </tr>
#                     <tr>
#                         <th>재무활동</th>
#                         <td class="bad-value">-10,979</td>
#                         <td class="bad-value">-12,739</td>
#                         <td>5,516</td>
#                         <td>3,944</td>
#                     </tr>
#                 </tbody>
#             </table>
#         </div>
#         <!-- 현금흐름표 END -->
#         <div class="a-line"></div>
#         <!-- 적정주가 계산 파트 -->
#         <div class="report-title">적정주가 계산</div>
#         <!-- 영업이익*PER 적정 주가 계산 -->
#         <div class="fs-head" style="margin-top: 20px">
#             <div class="fs-title">영업이익*PER</div>
#         </div>
#         <div class="standard-info">
#             <div class="info">
#                 <div class="info-cell">
#                     <div class="cell-head">영업이익</div>
#                     <div class="cell-desc">14,160억원</div>
#                 </div>

#                 <div class="info-cell">
#                     <div class="cell-head">기준</div>
#                     <div class="cell-desc">2023/12(E)</div>
#                 </div>
#             </div>
#             <div class="info-one">
#                 <div class="cell-head">현재주가</div>
#                 <div class="cell-desc">318,500원</div>
#             </div>
#         </div>
#         <div class="fs-div" style="margin-top: 10px">
#             <table class="fs-data">
#                 <thead>
#                     <th>구분</th>
#                     <th>PER (%)</th>
#                     <th>기업가치</th>
#                     <th>적정주가</th>
#                 </thead>
#                 <tbody>
#                     <tr>
#                         <th>올해 예상</th>
#                         <td>11.32</td>
#                         <td>160,291.2억원</td>
#                         <td>1,064,762원</td>
#                     </tr>
#                     <tr>
#                         <th>업종 평균</th>
#                         <td>11.32</td>
#                         <td>160,291.2억원</td>
#                         <td>1,064,762원</td>
#                     </tr>
#                     <tr>
#                         <th>별도 조정</th>
#                         <td>
#                             <input class="custom-input per-input" value="10" type="number" min="0" max="100" step="1" />
#                         </td>
#                         <td><span class="custom-value per-fair-value"></span></td>
#                         <td><span class="custom-value per-fair-price"></span></td>
#                     </tr>
#                 </tbody>
#             </table>
#         </div>
#         <!-- 영업이익*PER END -->
#         <!-- 영업이익*PER Chart -->
#         <div class="fs-chard-div">
#             <canvas class="fs-per-chart"></canvas>
#             <script>
#                 const perCanvas = document.querySelector('.fs-per-chart').getContext('2d');
#                 const nowValue = 318500;
#                 let calcPer = 0;
#                 const perChart = new Chart(perCanvas, {
#                     type: 'line',
#                     data: {
#                         labels: ['올해 예상', '업종 평균', '별도 조정'],
#                         datasets: [{
#                             label: '적정주가',
#                             data: [1064762, 1064762, calcPer],
#                             backgroundColor: 'rgba(16, 163, 127, 0.2)',
#                             borderColor: 'rgba(16, 163, 127, 1)',
#                             borderWidth: 1,
#                             type: 'bar'
#                         }]
#                     },
#                     options: {
#                         scales: {
#                             y: {
#                                 beginAtZero: true,
#                                 position: 'left',
#                                 ticks: {
#                                     callback: function(value, index, ticks) {
#                                         return `${value.toLocaleString()}원`;
#                                     }
#                                 }
#                             }
#                         },
#                         plugins: {
#                             legend: {
#                                 onClick: (e) => e.stopPropagation()
#                             },
#                             annotation: {
#                                 annotations: {
#                                     currentValue: {
#                                         type: 'line',
#                                         mode: 'horizontal',
#                                         scaleID: 'y',
#                                         value: nowValue,
#                                         borderColor: 'rgba(255, 127, 80, 1)',
#                                         borderWidth: 1.5,
#                                         label: {
#                                             content: '현재 주가',
#                                             enabled: true,
#                                             position: 'top',
#                                         }
#                                     },
#                                     label1: {
#                                         type: 'label',
#                                         yValue: nowValue,
#                                         content: ['현재 주가', `${nowValue.toLocaleString()}원`],
#                                         color: 'rgba(255, 127, 80, 1)',
#                                         font: {
#                                             size: 13,
#                                         }
#                                     }
#                                 }
#                             }
#                         }
#                     }
#                 });

#                 const perInput = document.querySelector('.per-input');
#                 const perHandler = (perValue) => {
#                     const fairValue = document.querySelector('.per-fair-value');
#                     const fairPrice = document.querySelector('.per-fair-price');

#                     if (parseInt(perValue) > 100) {
#                         perValue = 100;
#                     }

#                     const earning = 14160; // 영업이익
#                     const stockCnt = 1504186; // 발행주식수
#                     const per = parseFloat(perValue);

#                     // 기업가치
#                     let calcEarning = earning * per;
#                     let calcPrice = (calcEarning / stockCnt) * 10000000;

#                     calcEarning = calcEarning.toFixed(1);
#                     calcPrice = calcPrice.toFixed(0);

#                     if (isNaN(calcEarning)) {
#                         fairValue.innerHTML = '';
#                         fairPrice.innerHTML = '';
#                         calcPrice = 0;
#                     } else {
#                         fairValue.innerHTML = `${parseFloat(calcEarning).toLocaleString()}억원`;
#                         fairPrice.innerHTML = `${parseInt(calcPrice).toLocaleString()}원`;
#                     }

#                     calcPer = calcPrice;
#                     perChart.data.datasets[0].data[2] = calcPer;

#                     perChart.update();
#                 };

#                 perInput.addEventListener('input', (event) => {
#                     perHandler(event.target.value);
#                 });
#                 perHandler(10);
#             </script>
#         </div>
#         <!-- 영업이익*PER Chart END -->
#         <!-- SRIM 계산 -->
#         <div class="fs-head" style="margin-top: 20px">
#             <div class="fs-title">SRIM</div>
#         </div>
#         <div class="standard-info">
#             <div class="info-one">
#                 <div class="cell-head">발행주식수</div>
#                 <div class="cell-desc">402,044,203</div>
#             </div>
#             <div class="info-one">
#                 <div class="cell-head">자사주</div>
#                 <div class="cell-desc">6,962,511</div>
#             </div>

#             <div class="info" style="margin-top: 15px;">
#                 <div class="info-cell">
#                     <div class="cell-head">지배주주지분</div>
#                     <div class="cell-desc">393,379</div>
#                 </div>

#                 <div class="info-cell">
#                     <div class="cell-head">BBB-</div>
#                     <div class="cell-desc">10.85%</div>
#                 </div>
#                 <div class="info-one">
#                     <div class="cell-head">예상 ROE</div>
#                     <div class="cell-desc">19.33%</div>
#                 </div>
#                 <div class="info-one">
#                     <div class="cell-head">가중 ROE</div>
#                     <div class="cell-desc">13.02%</div>
#                 </div>
#             </div>
#         </div>
#         <div class="srim-page">
#             <div class="srim-roe-head">
#                 <div class="srim-roe roe-selected">예상 ROE</div>
#                 <div class="srim-roe">가중 ROE</div>
#                 <div class="srim-roe-info">기준 ROE 선택</div>
#             </div>
#             <div class="fs-div srim-result" style="margin-top: 10px">
#                 <table class="fs-data srim-table">
#                     <thead>
#                         <th>초과이익</th>
#                         <th>기업가치</th>
#                         <th>적정주가</th>
#                     </thead>
#                     <tbody>
#                         <tr>
#                             <th>이익 지속</th>
#                             <td>1700,830.97억</td>
#                             <td>177,389원</td>
#                         </tr>
#                         <tr>
#                             <th>10% 감소</th>
#                             <td>537,372.69억</td>
#                             <td>136,016원</td>
#                         </tr>
#                         <tr>
#                             <th>20% 감소</th>
#                             <td>479,884.13억</td>
#                             <td>121,465원</td>
#                         </tr>
#                         <tr>
#                             <th>30% 감소</th>
#                             <td>379,884.13억</td>
#                             <td>111,465원</td>
#                         </tr>
#                         <tr>
#                             <th>40% 감소</th>
#                             <td>279,884.13억</td>
#                             <td>91,465원</td>
#                         </tr>
#                         <tr>
#                             <th>50% 감소</th>
#                             <td>179,884.13억</td>
#                             <td>71,465원</td>
#                         </tr>
#                     </tbody>
#                 </table>

#                 <table class="fs-data srim-table post-fs-hidden">
#                     <thead>
#                         <th>초과이익</th>
#                         <th>기업가치</th>
#                         <th>적정주가</th>
#                     </thead>
#                     <tbody>
#                         <tr>
#                             <th>이익 지속</th>
#                             <td>1800,830.97억</td>
#                             <td>187,389원</td>
#                         </tr>
#                         <tr>
#                             <th>10% 감소</th>
#                             <td>547,372.69억</td>
#                             <td>146,016원</td>
#                         </tr>
#                         <tr>
#                             <th>20% 감소</th>
#                             <td>479,884.13억</td>
#                             <td>131,465원</td>
#                         </tr>
#                         <tr>
#                             <th>30% 감소</th>
#                             <td>379,884.13억</td>
#                             <td>111,465원</td>
#                         </tr>
#                         <tr>
#                             <th>40% 감소</th>
#                             <td>279,884.13억</td>
#                             <td>91,465원</td>
#                         </tr>
#                         <tr>
#                             <th>50% 감소</th>
#                             <td>179,884.13억</td>
#                             <td>71,465원</td>
#                         </tr>
#                     </tbody>
#                 </table>
#             </div>
#         </div>
#         <!-- SRIM END -->
#         <!-- SRIM Chart -->
#         <div class="fs-chard-div">
#             <canvas class="fs-srim-chart"></canvas>
#         </div>
#         <script>
#             // SRIM CHART
#             const srimCanvas = document.querySelector('.fs-srim-chart').getContext('2d');
#             const srimResults = [
#                 [267613, 369561, 405503, 425503, 455503, 485503],
#                 [485503, 455503, 425503, 405503, 369561, 267613]
#             ]
#             let setSRIM = srimResults[0];
#             const srimChart = new Chart(srimCanvas, {
#                 type: 'line',
#                 data: {
#                     labels: ['이익 지속', '10% 감소', '20% 감소', '30% 감소', '40% 감소', '50% 감소'],
#                     datasets: [{
#                         label: '적정주가',
#                         data: setSRIM,
#                         backgroundColor: 'rgba(16, 163, 127, 0.2)',
#                         borderColor: 'rgba(16, 163, 127, 1)',
#                         borderWidth: 1,
#                         type: 'bar'
#                     }]
#                 },
#                 options: {
#                     scales: {
#                         y: {
#                             beginAtZero: true,
#                             position: 'left',
#                             ticks: {
#                                 callback: function(value, index, ticks) {
#                                     return `${value.toLocaleString()}원`;
#                                 }
#                             }
#                         }
#                     },
#                     plugins: {
#                         legend: {
#                             onClick: (e) => e.stopPropagation()
#                         },
#                         annotation: {
#                             annotations: {
#                                 currentValue: {
#                                     type: 'line',
#                                     mode: 'horizontal',
#                                     scaleID: 'y',
#                                     value: nowValue,
#                                     borderColor: 'rgba(255, 127, 80, 1)',
#                                     borderWidth: 1.5,
#                                     label: {
#                                         content: '현재 주가',
#                                         enabled: true,
#                                         position: 'top',
#                                     }
#                                 },
#                                 label1: {
#                                     type: 'label',
#                                     yValue: nowValue,
#                                     content: ['현재 주가', `${nowValue.toLocaleString()}원`],
#                                     color: 'rgba(255, 127, 80, 1)',
#                                     font: {
#                                         size: 13,
#                                     }
#                                 }
#                             }
#                         }
#                     }
#                 }
#             });
#             // ROE 선택 기능
#             const roeBtn = document.querySelectorAll('.srim-roe');
#             const roeSelected = 'roe-selected';
#             const roeHandler = (index) => {
#                 const roeHead = document.querySelector('.srim-roe-head');
#                 const selectedRoe = roeHead.querySelector(`.${roeSelected}`);
#                 const roeTables = document.querySelectorAll('.srim-table');
#                 selectedRoe.classList.remove(roeSelected);

#                 roeBtn[index].classList.add(roeSelected);
#                 roeTables.forEach((table) => {
#                     table.classList.add('post-fs-hidden');
#                 });
#                 roeTables[index].classList.remove('post-fs-hidden');

#                 // Chart update
#                 setSRIM = srimResults[index];
#                 srimChart.data.datasets[0].data = setSRIM;

#                 srimChart.update();
#             };

#             roeBtn.forEach((roe, index) => {
#                 roe.addEventListener('click', () => {
#                     roeHandler(index);
#                 });
#             });
#         </script>
#     </div>
#     <!-- SRIM Chart END -->
#     <div class="a-line"></div>
# </div>
# <!-- END -->
# '''